# Experiment 007: Egortrushin Lattice + SA Approach

Implement the egortrushin approach that optimizes TRANSLATION PARAMETERS (dx, dy) and angles,
not just tree positions. This explores DIFFERENT solution basins.

In [1]:
import math
import random
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 25
scale_factor = Decimal("1e15")

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal("0.0") * scale_factor, tip_y * scale_factor),
            (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

    def get_params(self):
        return self.center_x, self.center_y, self.angle

    def set_params(self, center_x, center_y, angle):
        self.__init__(str(center_x), str(center_y), str(angle))

    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print("ChristmasTree class defined!")

ChristmasTree class defined!


In [2]:
def calculate_score(trees):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    score = max(max_x - min_x, max_y - min_y) ** 2 / len(trees)
    return score

def has_collision(trees):
    if len(trees) <= 1:
        return False
    for i, tree1 in enumerate(trees):
        for j, tree2 in enumerate(trees):
            if i < j:
                if tree1.polygon.intersects(tree2.polygon) and not tree1.polygon.touches(tree2.polygon):
                    return True
    return False

print("Helper functions defined!")

Helper functions defined!


In [3]:
class SimulatedAnnealing:
    """SA that optimizes translation parameters (dx, dy) and angles for lattice packing."""
    def __init__(self, trees, nt, Tmax, Tmin, nsteps, nsteps_per_T, cooling, alpha, n,
                 position_delta, angle_delta, delta1, random_state, log_freq):
        self.trees = trees  # 2 initial trees
        self.nt = nt  # [ncols, nrows]
        self.Tmax = Tmax
        self.Tmin = Tmin
        self.nsteps = nsteps
        self.nsteps_per_T = nsteps_per_T
        self.cooling = cooling
        self.alpha = alpha
        self.n = n  # target number of trees
        self.position_delta = position_delta
        self.angle_delta = angle_delta
        self.delta1 = delta1
        self.log_freq = log_freq
        random.seed(random_state)

    def perturb_tree(self, tree):
        old_x, old_y, old_angle = tree.get_params()
        dx = Decimal(str(random.uniform(-self.position_delta, self.position_delta)))
        dy = Decimal(str(random.uniform(-self.position_delta, self.position_delta)))
        dangle = Decimal(str(random.uniform(-self.angle_delta, self.angle_delta)))
        new_x = old_x + dx
        new_y = old_y + dy
        new_angle = (old_angle + dangle) % 360
        tree.set_params(new_x, new_y, new_angle)
        return old_x, old_y, old_angle

    def get_length(self, current_trees):
        xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in current_trees])
        min_x, min_y = xys.min(axis=0)
        max_x, max_y = xys.max(axis=0)
        return max(max_x - min_x, max_y - min_y)

    def translate_trees(self, trees, dx, dy):
        """Create grid of trees by translating the initial 2 trees."""
        result = []
        for i in range(self.nt[0]):
            for j in range(self.nt[1]):
                for tree in trees:
                    if len(result) >= self.n:
                        break
                    new_x = tree.center_x + Decimal(str(i * dx))
                    new_y = tree.center_y + Decimal(str(j * dy))
                    result.append(ChristmasTree(str(new_x), str(new_y), str(tree.angle)))
                if len(result) >= self.n:
                    break
            if len(result) >= self.n:
                break
        return result[:self.n]

    def solve(self):
        # Initialize with some dx, dy values
        dx = 0.6
        dy = 0.6
        
        current_trees = self.translate_trees(self.trees, dx, dy)
        current_length = self.get_length(current_trees)
        best_length = current_length
        best_trees = [t.clone() for t in current_trees]
        best_dx, best_dy = dx, dy
        best_base_trees = [t.clone() for t in self.trees]
        
        T = self.Tmax
        step = 0
        
        while T > self.Tmin and step < self.nsteps:
            for _ in range(self.nsteps_per_T):
                step += 1
                if step >= self.nsteps:
                    break
                
                # Randomly choose what to perturb
                choice = random.randint(0, 3)
                
                if choice == 0:  # Perturb dx
                    old_dx = dx
                    dx += random.uniform(-self.delta1, self.delta1)
                    dx = max(0.3, min(1.5, dx))  # Keep dx in reasonable range
                    new_trees = self.translate_trees(self.trees, dx, dy)
                    if has_collision(new_trees):
                        dx = old_dx
                        continue
                    new_length = self.get_length(new_trees)
                    delta = new_length - current_length
                    if delta < 0 or random.random() < math.exp(-delta / T):
                        current_trees = new_trees
                        current_length = new_length
                        if new_length < best_length:
                            best_length = new_length
                            best_trees = [t.clone() for t in new_trees]
                            best_dx, best_dy = dx, dy
                            best_base_trees = [t.clone() for t in self.trees]
                    else:
                        dx = old_dx
                        
                elif choice == 1:  # Perturb dy
                    old_dy = dy
                    dy += random.uniform(-self.delta1, self.delta1)
                    dy = max(0.3, min(1.5, dy))
                    new_trees = self.translate_trees(self.trees, dx, dy)
                    if has_collision(new_trees):
                        dy = old_dy
                        continue
                    new_length = self.get_length(new_trees)
                    delta = new_length - current_length
                    if delta < 0 or random.random() < math.exp(-delta / T):
                        current_trees = new_trees
                        current_length = new_length
                        if new_length < best_length:
                            best_length = new_length
                            best_trees = [t.clone() for t in new_trees]
                            best_dx, best_dy = dx, dy
                            best_base_trees = [t.clone() for t in self.trees]
                    else:
                        dy = old_dy
                        
                else:  # Perturb one of the base trees
                    tree_idx = random.randint(0, len(self.trees) - 1)
                    old_params = self.perturb_tree(self.trees[tree_idx])
                    new_trees = self.translate_trees(self.trees, dx, dy)
                    if has_collision(new_trees):
                        self.trees[tree_idx].set_params(*old_params)
                        continue
                    new_length = self.get_length(new_trees)
                    delta = new_length - current_length
                    if delta < 0 or random.random() < math.exp(-delta / T):
                        current_trees = new_trees
                        current_length = new_length
                        if new_length < best_length:
                            best_length = new_length
                            best_trees = [t.clone() for t in new_trees]
                            best_dx, best_dy = dx, dy
                            best_base_trees = [t.clone() for t in self.trees]
                    else:
                        self.trees[tree_idx].set_params(*old_params)
            
            T *= self.cooling
            
            if step % self.log_freq == 0:
                print(f"Step {step}, T={T:.6f}, best_length={best_length:.6f}")
        
        score = best_length ** 2 / self.n
        return score, best_trees

print("SimulatedAnnealing class defined!")

SimulatedAnnealing class defined!


In [4]:
# Load baseline for comparison
baseline_path = '/home/code/experiments/001_baseline/santa-2025.csv'
df_baseline = pd.read_csv(baseline_path, dtype=str)

def load_baseline_configs(df):
    configs = {}
    for n in range(1, 201):
        prefix = f'{n:03d}_'
        rows = df[df['id'].str.startswith(prefix)]
        trees = []
        for _, row in rows.iterrows():
            x = str(row['x']).replace('s', '')
            y = str(row['y']).replace('s', '')
            deg = str(row['deg']).replace('s', '')
            trees.append(ChristmasTree(x, y, deg))
        configs[n] = trees
    return configs

baseline_configs = load_baseline_configs(df_baseline)
baseline_scores = {n: calculate_score(baseline_configs[n]) for n in range(1, 201)}
baseline_total = sum(baseline_scores.values())
print(f"Baseline total score: {baseline_total:.6f}")

Baseline total score: 70.676102


In [ ]:
# Run lattice + SA for target N values
config = {
    "params": {
        "Tmax": 1.0,
        "Tmin": 0.0001,
        "nsteps": 50000,
        "nsteps_per_T": 100,
        "cooling": 0.995,
        "alpha": 0.5,
        "position_delta": 0.05,
        "angle_delta": 5.0,
        "delta1": 0.02,
        "random_state": 42,
        "log_freq": 10000,
    }
}

# Target N values and their grid dimensions
lattice_targets = [
    (72, [4, 9]),    # 4x9 grid with 2 trees per cell = 72
    (100, [5, 10]),  # 5x10 = 100
    (144, [6, 12]),  # 6x12 = 144
    (156, [6, 13]),  # 6x13 = 156
    (196, [7, 14]),  # 7x14 = 196
]

print("Running lattice + SA optimization...")
lattice_results = {}

for n, nt in lattice_targets:
    print(f"\n=== N={n}, grid={nt} ===")
    
    # Initial 2 trees
    initial_trees = [
        ChristmasTree("0", "0", "0"),
        ChristmasTree("0.4", "0.4", "180")
    ]
    
    config["params"]["nt"] = nt
    config["params"]["n"] = n
    
    sa = SimulatedAnnealing(initial_trees, **config["params"])
    score, trees = sa.solve()
    
    baseline_score_n = baseline_scores[n]
    print(f"Baseline score: {baseline_score_n:.6f}")
    print(f"Lattice+SA score: {score:.6f}")
    
    if score < baseline_score_n:
        print(f"IMPROVED by {baseline_score_n - score:.6f}!")
        lattice_results[n] = trees
    else:
        print(f"No improvement (baseline is better)")

In [ ]:
# Calculate final score
final_configs = {n: list(baseline_configs[n]) for n in range(1, 201)}
for n, trees in lattice_results.items():
    final_configs[n] = trees

final_scores = {n: calculate_score(final_configs[n]) for n in range(1, 201)}
final_total = sum(final_scores.values())

print(f"\n=== RESULTS ===")
print(f"Baseline total: {baseline_total:.6f}")
print(f"Final total: {final_total:.6f}")
print(f"Improvement: {baseline_total - final_total:.6f}")

In [ ]:
# Save submission
def to_str(x):
    return f"s{float(x):.18f}"

rows = []
for n in range(1, 201):
    trees = final_configs[n]
    for i, tree in enumerate(trees):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': to_str(tree.center_x),
            'y': to_str(tree.center_y),
            'deg': to_str(tree.angle)
        })

submission_df = pd.DataFrame(rows)
submission_df.to_csv('/home/submission/submission.csv', index=False)
submission_df.to_csv('/home/code/experiments/007_lattice_sa/submission.csv', index=False)
print(f"Saved submission")

print(f"\n=== FINAL SCORE: {final_total:.6f} ===")